<a href="https://colab.research.google.com/github/wolfgames/FlowiseAI-Railway/blob/main/voicegen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Enter credentials to access a Casescope/Casemaker Apis.

In [ ]:
from getpass import getpass

email = input("Email: ")
password = getpass("Password: ")

Authenticating and getting auth token

In [ ]:
import requests
import pprint
from IPython.display import Markdown as md


response = requests.post("https://auth.casescope.com/auth/signin", {"email": email, "password": password})

print(response.status_code)
auth_data = response.json()
access_token = auth_data['accessToken']
pp = pprint.PrettyPrinter(indent=1)
#pp.pprint(auth_data)
print(f"Auth Token: {access_token}")


Requesting prompt "case_common_info" generating a Case.

In [ ]:
casescope_url = "https://api.casescope.com"
headers = {"Authorization": f"Bearer {access_token}"}

def get_prompt(prompt_id: int):
  response = requests.get(f"{casescope_url}/ai_prompt/{prompt_id}", headers=headers)
  data = response.json()
  prompt = data["prompt"]
  print(f"Prompt: {prompt}")
  return prompt



Generate case common info

In [ ]:
def generate_text(prompt: str, parameters={}):
  data = {"parameters": parameters, "prompt": prompt}
  print(f"Json: {data}")
  response = requests.post(f"{casescope_url}/openai/generate", json=data, headers=headers)
  data = response.json()
  return data

prompt = get_prompt(2)
case_common_info = generate_text(prompt)
# CaseMaker data format required for persisting data
print(case_common_info)

Generate case synopsis

In [ ]:
prompt = get_prompt(3)
case_synopsis = generate_text(prompt, {"case_name": case_common_info["caseName"], "crime_type": case_common_info["crimeType"], "crime_location": case_common_info["caseLocation"], "suspects_number": case_common_info["numberSuspects"]})
# CaseMaker data format required for persisting data
print(case_synopsis)

Generate case introduction

In [ ]:
prompt = get_prompt(4)
case_intro = generate_text(prompt, {"case_name": case_common_info["caseName"], "crime_type": case_common_info["crimeType"], "crime_location": case_common_info["caseLocation"], "suspects_number": case_common_info["numberSuspects"], "synopsis": case_synopsis})
# CaseMaker data format required for persisting data
print(case_intro)

Generate Writer's Markdown Synopsis

In [ ]:
prompt = get_prompt(25)
case_synopsis_md = generate_text(prompt, {"synopsis": case_synopsis})
# CaseMaker data format required for persisting data
md(case_synopsis_md)

Case narrative

In [ ]:
prompt = get_prompt(28)
case_narrative = generate_text(prompt, { "case_synopsis_md": case_synopsis_md, })
# CaseMaker data format required for persisting data
pp.pprint(case_narrative)

Generate list of characters in the case

In [ ]:
character_case_relation = '"Culprit" || "Witness" || "Suspect" || "Victim" || "Incidental"'
prompt = get_prompt(29)
case_characters = generate_text(prompt, { "case_synopsis_md": case_synopsis_md, "role_options": character_case_relation,  "case_narrative": case_narrative})

# CaseMaker data format required for persisting data
pp.pprint(case_characters)

# NEEDS UPDATING

In [ ]:
# Mapping characters data
# mapped_characters = list(map(lambda c: {"age": c["character"]["age"], "firstName": c["character"]["firstName"], "lastName": c["character"]["lastName"], "sex": c["character"]["sex"], "characterOccupation": c["character"]["occupation"], "description": c["character"]["description"], "caseRelation": c["characterCaseRelation"]}, case_characters))
# pp.pprint(mapped_characters)

Generate case events

In [ ]:
prompt = get_prompt(22)
case_events = generate_text(prompt, {
    "synopsis": case_synopsis_md,
    "charactersData": case_characters,
    "narrative": case_narrative
    })
# CaseMaker data format required for persisting data
pp.pprint(case_events)

add causal events integration here

# Causal Graph Addition

Generate event relations

In [ ]:
prompt = get_prompt(23)
events_relations = generate_text(prompt, {
    "narrative": case_narrative,
    "events": case_events
    })
# CaseMaker data format required for persisting data
pp.pprint(events_relations)

Generate Concept Tree

In [ ]:
prompt = get_prompt(30)
case_tree = generate_text(prompt, {
    "case_synopsis_md": case_synopsis_md,
    "case_characters": case_characters
    })
# CaseMaker data format required for persisting data
md(case_tree)

In [ ]:
prompt = get_prompt(32)
case_solution = generate_text(prompt, {
    "case_synopsis_md": case_synopsis_md,
    "case_characters": case_characters,
    "case_tree": case_tree
    })
# CaseMaker data format required for persisting data
md(case_solution)

In [ ]:
prompt = get_prompt(33)
evidence_tier3 = generate_text(prompt, {
    "case_synopsis_md": case_synopsis_md,
    "case_characters": case_characters,
    "case_tree": case_tree,
    "case_events": case_events,
    "case_solution": case_solution
    })
# CaseMaker data format required for persisting data
pp.pprint(evidence_tier3)

In [ ]:
prompt = get_prompt(35)
evidence_tier2 = generate_text(prompt, {
    "case_synopsis_md": case_synopsis_md,
    "case_characters": case_characters,
    "case_tree": case_tree,
    "case_events": case_events,
    "case_solution": case_solution,
    "evidence_tier3": evidence_tier3
    })
# CaseMaker data format required for persisting data
pp.pprint(evidence_tier2)

In [ ]:
prompt = get_prompt(36)
evidence_tier1 = generate_text(prompt, {
    "case_synopsis_md": case_synopsis_md,
    "case_characters": case_characters,
    "case_tree": case_tree,
    "case_events": case_events,
    "case_solution": case_solution,
    "evidence_tier3": evidence_tier3,
    "evidence_tier2": evidence_tier2
    })
# CaseMaker data format required for persisting data
pp.pprint(evidence_tier1)

In [ ]:
prompt = get_prompt(38)
evidence_tier0 = generate_text(prompt, {
    "case_synopsis_md": case_synopsis_md,
    "case_characters": case_characters,
    "case_tree": case_tree,
    "case_events": case_events,
    "case_solution": case_solution,
    "evidence_tier3": evidence_tier3,
    "evidence_tier2": evidence_tier2,
    "evidence_tier1": evidence_tier1,
    })
# CaseMaker data format required for persisting data
pp.pprint(evidence_tier0)

In [ ]:
prompt = get_prompt(39)
prosecutor_validator = generate_text(prompt, {
    "evidence_tier3": evidence_tier3,
    "evidence_tier2": evidence_tier2,
    "evidence_tier1": evidence_tier1,
    "evidence_tier0": evidence_tier0,
    })
# CaseMaker data format required for persisting data
md(prosecutor_validator)

# NEEDS UPDATING
Here is how output of evidences are being mapped for UI / Database needs.

In [ ]:
import uuid

tiers = list(map(lambda t: next(iter(t.keys())), evidences["tiers"]))
tiers.append('Solutions')

cards = []

for tier in evidences["tiers"]:
  for key in tier.keys():
    for card in tier[key]:
      mapped_card = {"card_type": card["card_type"], "card_text": card["card_text"], "tier": key, "uid": str(uuid.uuid4().hex)}
      cards.append(mapped_card)

for solution in evidences["solutions"]:
  for key, value in solution.items():
    mapped_card = {"card_type": key, "card_text": value, "tier": "Solution", "uid": str(uuid.uuid4().hex)}
    cards.append(mapped_card)

casemaker_evidence_list = list(
    map(
        lambda card: {
            "uid": card["uid"],
            "name": card["card_type"],
            "description": '',
            "type": 'TEXT',
            "tier": card["tier"],
            "data": {
                "value": card["card_text"],
                "description": ''
            },
            "propertyGenerationStatus": 'INITIAL',
            "propertiesType": 'NONE',
            "relations": [],
            "properties": []
        },
        cards
    )
)

casemaker_evidence_board = { # CaseMaker data format required for persisting data
    "characteristics": casemaker_evidence_list,
    "tiers": tiers
}
pp.pprint(casemaker_evidence_board)